In [1]:
#File Imports
import os
from datetime import date
import zipfile
from shutil import copytree
import json
from os.path import exists

#GUI Imports For File Selection
import tkinter
from tkinter import filedialog
from tkinter import Tk

#Imports for Notion Updates
import requests

In [2]:
#Check Configuration
#file_exists = exists('config.json') #does config file exist? 
file_exists = exists('config.json') #does config file exist? 
home = os.getcwd()
if file_exists:
    with open('config.json') as f:
        #load properties
        config = json.load(f) 
    #check if all paths exist before moving forward
    for c in config: 
        #if path does not exist, print it and quit the program
        if not (exists(config[c])):
            print('{} : {} does\'t exist'.format(c, config[c]))
            quit()
else:
    #create an empty configuration file for easy user entry. 
    print('Populate the configuration file! with forward slashes!')
    default_config = {
        'local_pkgs':'',
        'local_setup':'',
        'private_pkgs':'',
        'public_pkgs':'',
        'local_downloads':'',
        'private_ifc':'',
        'public_ifc':'',
        'private_pos':'',
        'private_prd':'',
        'private_dd':''
    }
    with open('config.json', 'w', encoding='utf8') as f:
        json.dump(default_config, f, ensure_ascii = True)

In [3]:
#Alpha Prefix
prefix = 'OHIEX-R-B0001'
stages = ['IFC','POS','PRD','30','60','90'] #default stages
print('Prompt will ask for name, stage, description, and owner.')
pkg_name = '{}-{}'.format(prefix, input('Name: ').strip().upper())

print('Stages: {}'.format(stages))
pkg_stage = input('Stage: ').strip().upper()
while(pkg_stage not in stages):
    print('Stages: {}'.format(stages))
    print('Pick Preset Stage!')
    pkg_stage = input('Stage: ').strip().upper()

pkg_desc = input('Description: ').strip().upper()
pkg_owner = input('Owner: ').strip().upper()

pkg_info = {
    'pkg_name' : pkg_name,
    'pkg_stage' : pkg_stage,
    'pkg_desc' : pkg_desc,
    'pkg_date' : date.today().isoformat(),
    'pkg_owner' : pkg_owner
}
print('PACKAGE INFO:')
print('-'*20)
for key in pkg_info:
    print('{}: {}'.format(key,pkg_info[key]))
print('-'*20)

Prompt will ask for name, stage, description, and owner.
Stages: ['IFC', 'POS', 'PRD', '30', '60', '90']
PACKAGE INFO:
--------------------
pkg_name: OHIEX-R-B0001-CPK03-R2
pkg_stage: 90
pkg_desc: CSA_CLEAN_MEP
pkg_date: 2021-11-12
pkg_owner: MAX PARKER
--------------------


In [4]:
#Select original files
application_window = tkinter.Tk()
pkg_original = filedialog.askopenfilename(initialdir=config['local_downloads'], title="Please select package zip file:")
pkg_email = filedialog.askopenfilename(initialdir=config['local_downloads'], title="Please select package email file:")
application_window.destroy()

In [5]:
is_zip = pkg_original[-4:]=='.zip'
is_msg = pkg_email[-4:]=='.msg'

In [6]:
#prepare package 
#TODO logic to make sure this valid
if not (is_zip and is_msg):
    print('Incorrect files!')
    quit()

pkg_filename = '{}-{} - {}'.format(pkg_info['pkg_name'],pkg_info['pkg_stage'],pkg_info['pkg_date'])
pkg_filename_bb = 'A - {} - {} - {} - {}'.format(pkg_info['pkg_name'],pkg_info['pkg_stage'],pkg_info['pkg_desc'],pkg_info['pkg_date'])

#new folder with package name
os.chdir(config['local_downloads'])
os.mkdir(pkg_filename)

#unzip contents to that folder
with zipfile.ZipFile(pkg_original, 'r') as zip_ref:
    zip_ref.extractall(pkg_filename)

#move zip file folder with proper name
os.rename(pkg_original, os.path.join(pkg_filename, pkg_filename+'.zip')) #move

In [7]:
#relative to downloads folder because thats where os is. 
#yes the last three are the same for clarity of flow and reading. 
if pkg_info['pkg_stage']==stages[0]:
    copytree(pkg_filename, os.path.join(config['public_ifc'], pkg_filename))
    os.rename(pkg_email, os.path.join(pkg_filename,pkg_filename+'.msg')) #move
    copytree(pkg_filename, os.path.join(config['private_ifc'], pkg_filename))
    # TODO: Create a folder in SBN and Bids for package name with standard folders.
elif pkg_info['pkg_stage']==stages[1]:
    os.rename(pkg_email, os.path.join(pkg_filename,pkg_filename+'.msg')) #move
    copytree(pkg_filename, os.path.join(config['private_pos'], pkg_filename))
elif pkg_info['pkg_stage']==stages[2]:
    os.rename(pkg_email, os.path.join(pkg_filename,pkg_filename+'.msg')) #move
    copytree(pkg_filename, os.path.join(config['private_prd'], pkg_filename))
elif pkg_info['pkg_stage']==stages[3]:
    os.rename(pkg_email, os.path.join(pkg_filename,pkg_filename+'.msg')) #move
    copytree(pkg_filename, os.path.join(config['private_dd'], pkg_filename))
elif pkg_info['pkg_stage']==stages[4]:
    os.rename(pkg_email, os.path.join(pkg_filename,pkg_filename+'.msg')) #move
    copytree(pkg_filename, os.path.join(config['private_dd'], pkg_filename))
elif pkg_info['pkg_stage']==stages[5]:
    os.rename(pkg_email, os.path.join(pkg_filename,pkg_filename+'.msg')) #move
    copytree(pkg_filename, os.path.join(config['private_dd'], pkg_filename))
else:
    print('copy did not happen, please perform manually')

In [8]:
with open(os.path.join(home,'new_package.json')) as json_file:
    data = json.load(json_file) #load templated file for new package item

In [9]:
data['properties']['Name']['title'][0]['text']['content'] = '{}-{}'.format(pkg_info['pkg_name'], pkg_info['pkg_stage'])

In [10]:
database_id = 'c674b9c21aa348a89a37201a65f42070'
token = 'secret_NyIXoFGzUS5k681N96D2G1irDIF5hpADeNfMJNMUuFp'

In [11]:
createurl ='https://api.notion.com/v1/pages'
cheaders={
    'Authorization' : 'Bearer ' + token,
    'Content-Type': 'application/json',
    'Notion-Version' : '2021-08-16'
}
r=requests.request('POST', createurl, headers = cheaders, data = json.dumps(data))
print('Status of Notion post: {}'.format(r.status_code))

Status of Notion post: 200


In [12]:
os.chdir(config['local_downloads'])
os.listdir()

['desktop.ini',
 'OHIEX-R-B0001-CPK03-R1-IFC - 2021-11-02',
 'revblock explain.pdf']

In [15]:
#preparing local version
os.chdir(pkg_filename)
wrk_filename = 'work-{}'.format(date.today().isoformat())
os.mkdir(wrk_filename) #make work folder
os.chdir(wrk_filename)
with open(pkg_filename_bb+".pdf",'w') as bbname:
    bbname.write('work folder created. filename is a reference for easy naming when saving pdf.')

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'OHIEX-R-B0001-CPK03-R1-IFC - 2021-11-02'

In [13]:
#move to documents #PERMISSION DENIED ERROR
os.chdir(config['local_downloads'])
os.rename(pkg_filename,os.path.join(config['local_pkgs'],pkg_filename)) #move

PermissionError: [WinError 5] Access is denied: 'OHIEX-R-B0001-CPK03-R5-60-2021-10-15' -> 'C:/Users/maxpar/OneDrive - JEDunn/Documents/local_project/alpha/packages\\OHIEX-R-B0001-CPK03-R5-60-2021-10-15'

In [15]:
#Add to ownership file. #add entry logic here for dates, stages, etc. Change status when updated to maintained, not posted. 
owners = pd.read_excel(config['owner_doc'], index_col=None)

PermissionError: [Errno 13] Permission denied: 'C:/Users/maxpar/JEDunn/Project-21029900-Intel Project Alphabet - JE Dunn - Private/Preconstruction (Pre-Award)/Package Maintenance'

In [ ]:
owners.append([[pkg_info['pkg_name'],pkg_info['pkg_stage'],pkg_info['pkg_owner'],None,None,pkg_info['pkg_date'],'AUTO UPDATE BY BYTE OF PY BOT :)']])